In [1]:
# %load_ext iminizinc
%load_ext autoreload
%autoreload 2

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
# import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
from decimal import *
getcontext().prec = 6
import seaborn as sns
from scipy.signal import find_peaks, peak_widths
import pickle
import re


import sys, os
sys.path.append(os.path.abspath('..'))
from research.pkg.lib import *


# import mplfinance as mpf
%matplotlib inline

# plt.rcParams['lines.linewidth'] = 0.5 



In [2]:

# Machine learning libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.linear_model import Ridge, Lasso, LinearRegression


from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [3]:
pair = 'BTC_USDT'
timeframe = '4h'

exchange = 'binance'
odf = pd.read_json(f'../freq-user-data/data/{exchange}/{pair}-{timeframe}.json'
# exchange = 'kucoin'
# odf = pd.read_json(f'../../freq-user-data/data/{exchange}/futures/{pair}-{timeframe}-futures.json'
).dropna().set_axis(['timestamp', 'open', 'high', 'low', 'close', 'volume'], axis=1
).assign(dtime=lambda x: pd.to_datetime(x['timestamp'], unit='ms', utc=False)
).set_index('dtime').sort_index()

print(odf.shape)
odf.head(4)

(12676, 6)


,timestamp,open,high,low,close,volume
dtime,,,,,,
2017-08-17 04:00:00,1502942400000,4261.48,4349.99,4261.32,4349.99,82.088865
2017-08-17 08:00:00,1502956800000,4333.32,4485.39,4333.32,4427.30,63.619882
2017-08-17 12:00:00,1502971200000,4436.06,4485.39,4333.42,4352.34,174.562001
2017-08-17 16:00:00,1502985600000,4352.33,4354.84,4200.74,4325.23,225.109716


In [13]:
### Load it all. Period.

idf = odf.copy().ffill()

# body low and highs
idf['body_high'] = idf[['open', 'close']].max(axis=1)
idf['body_low'] = idf[['open', 'close']].min(axis=1)

# log return
idf['lret'] = idf.close.divide(idf.close.shift()).apply(np.log)
# candle price direction, up or down (1,-1)
idf['candle_dir'] = (1 - 2*(idf.lret < 0)).astype(int)
# number of down candles in last n periods
for per in [2,4,6,12]:
    idf[f'n_down_{per}'] = (idf.candle_dir < 0).astype(int).rolling(per).sum().astype('Int64')
    idf[f'n_up_{per}'] = (idf.candle_dir > 0).astype(int).rolling(per).sum().astype('Int64')
# next dir
idf['next_dir'] = idf.candle_dir.shift(-1)
# future returns for close and max/min high/low
# future_periods = [1,2,3,6]
# for fu in future_periods:
#     idf[f'flretC_{fu}'] = idf.close.shift(-fu).divide(idf.close).apply(np.log)
#     idf[f'flretH_{fu}'] = idf.high.rolling(fu).max().shift(-fu).divide(idf.close).apply(np.log)
#     idf[f'flretL_{fu}'] = idf.high.rolling(fu).max().shift(-fu).divide(idf.close).apply(np.log)

idf.to_csv('test.csv')
idf = idf.dropna()
idf

,timestamp,open,high,low,close,volume,body_high,body_low,lret,candle_dir,n_down_2,n_up_2,n_down_4,n_up_4,n_down_6,n_up_6,n_down_12,n_up_12,next_dir
dtime,,,,,,,,,,,,,,,,,,,
2017-08-19 00:00:00,1503100800000,4108.37,4184.69,4084.28,4138.55,108.510448,4138.55,4108.37,0.007319,1,1,1,3,1,3,3,6,6,-1.0
2017-08-19 04:00:00,1503115200000,4138.50,4156.00,3933.21,4033.47,112.752079,4138.50,4033.47,-0.025718,-1,1,1,3,1,4,2,7,5,-1.0
2017-08-19 08:00:00,1503129600000,4033.47,4082.25,3850.00,3957.60,77.256042,4033.47,3957.60,-0.018989,-1,2,0,3,1,5,1,8,4,1.0
2017-08-19 12:00:00,1503144000000,3945.12,4100.00,3928.89,3972.05,47.515578,3972.05,3945.12,0.003645,1,1,1,2,2,4,2,7,5,1.0
2017-08-19 16:00:00,1503158400000,4036.19,4103.92,3953.40,4076.12,25.925056,4076.12,4036.19,0.025863,1,0,2,2,2,3,3,6,6,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-01 12:00:00,1685620800000,26892.95,26981.34,26782.53,26894.48,6481.675460,26894.48,26892.95,0.000057,1,0,2,1,3,1,5,6,6,-1.0
2023-06-01 16:00:00,1685635200000,26894.47,27175.00,26656.00,26862.68,8974.247130,26894.47,26862.68,-0.001183,-1,1,1,1,3,2,4,7,5,-1.0
2023-06-01 20:00:00,1685649600000,26862.68,26942.98,26713.34,26817.93,3659.356110,26862.68,26817.93,-0.001667,-1,2,0,2,2,3,3,7,5,1.0


In [6]:
(idf.candle_dir < 0).astype(int)#.rolling(per).sum().abs()

dtime
2017-08-19 00:00:00    0
2017-08-19 04:00:00    1
2017-08-19 08:00:00    1
2017-08-19 12:00:00    0
2017-08-19 16:00:00    0
                      ..
2023-05-31 16:00:00    0
2023-05-31 20:00:00    0
2023-06-01 00:00:00    1
2023-06-01 04:00:00    0
2023-06-01 08:00:00    0
Name: candle_dir, Length: 12659, dtype: int64